In [0]:
!pip install -U -q kaggle

In [2]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
%matplotlib inline

# Description
The dataset is refer to [this notebook](https://www.kaggle.com/mahmudulhasanshauqi/deep-learning-with-python-chapter-2-ex-1) in kaggle.

In [0]:
# !mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [10]:
!kaggle datasets download -d mustafaali96/mnist -p /content/kaggle

 46% 5.00M/11.0M [00:00<00:00, 35.2MB/s]
100% 11.0M/11.0M [00:00<00:00, 55.6MB/s]


In [13]:
!unzip /content/kaggle/mnist.zip

Archive:  /content/kaggle/mnist.zip
  inflating: mnist.npz               


In [15]:
!ls ./

adc.json  kaggle  mnist.npz  sample_data


In [16]:
data = np.load('./mnist.npz')
print(data.keys())

['x_test', 'x_train', 'y_train', 'y_test']


In [19]:
print(data['x_test'].shape)
print(data['x_train'].shape)
print(data['y_test'].shape)
print(data['y_train'].shape)

(10000, 28, 28)
(60000, 28, 28)
(10000,)
(60000,)


In [0]:
# shuffle data
from random import shuffle
x_train = data['x_train']
y_train = data['y_train']
idx_list = [i for i in range(x_train.shape[0])]
shuffle(idx_list)
x_train_random = x_train[idx_list,:,:]
y_train_random = y_train[idx_list]
y_onehot = np.zeros([x_train.shape[0],y_train.max()+1],'int')
for i in range(x_train.shape[0]):
    y_onehot[i][y_train_random[i]]=1
# define the batch size
batch_size = 60
def get_data():
    for i in np.arange(0,x_train.shape[0],batch_size):
        yield (x_train_random[i:i+batch_size,:,:], y_onehot[i:i+batch_size,:])

In [0]:
# shuffle data
from random import shuffle
x_test = data['x_test']
y_test = data['y_test']
idx_list = [i for i in range(x_test.shape[0])]
shuffle(idx_list)
x_test_random = x_test[idx_list,:,:]
y_test_random = y_test[idx_list]
y_onehottest = np.zeros([x_test.shape[0],y_test.max()+1],'int')
for i in range(x_test.shape[0]):
    y_onehottest[i][y_test_random[i]]=1
# define the batch size
batch_size = 100
def get_test():
    for i in np.arange(0,x_test.shape[0],batch_size):
        yield (x_test_random[i:i+batch_size,:,:], y_onehottest[i:i+batch_size,:])

# Define Graph

In [0]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x,W)+b)

In [0]:
y_ = tf.placeholder("float", [None, 10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

In [0]:
train = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [0]:
init = tf.global_variables_initializer()

# Test

In [0]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

# Define Session
training happen only in a session

In [87]:
with tf.Session() as sess:
    sess.run(init)
    for (xs, ys) in get_data():
        sess.run(train, feed_dict={x:xs.reshape(60,784), y_:ys})
#     print(W.eval())
    for (xss, yss) in get_test():
        print(sess.run(accuracy, feed_dict={x: xss, y_: yss}))

ValueError: ignored